In [ ]:
import os
import cv2
import pandas as pd
import numpy as np
import PIL
import matplotlib.pyplot as plt
from tqdm import tqdm

import torchvision.transforms as transforms
import torchvision.models as models
import torch
from torch import nn
from torch.utils.data import TensorDataset, DataLoader

In [ ]:
os.listdir('/kaggle/input/emotion/train')

In [ ]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

In [ ]:
def dataloader(datapath='/kaggle/input/emotion', mode='train'):
#     labels = {'angry':0, 'happy':1, 'neutral':2, 'sad':3, 'surprised':4}
    labels = {'angry':0, 'happy':1, 'neutral':2}
    classes = []
    imgs = []
    print(f"mode {mode}")
    for label in labels.keys():
        dir_path = os.path.join(datapath, mode, label)
        print(f'{label} : {len(os.listdir(dir_path))}')
        for img_name in os.listdir(dir_path):
            img_path = os.path.join(dir_path, img_name)
            img_pil = PIL.Image.open(img_path)
            img = img_pil.convert('RGB')
            img = transform(img)
            imgs.append(img)
            classes.append(labels[label])
    imgs = torch.stack(imgs)
    classes = torch.LongTensor(classes)

    return imgs, classes

In [ ]:
x_train, y_train = dataloader(mode='train')
x_valid, y_valid = dataloader(mode='test')
print(x_train.shape, x_valid.shape)

In [ ]:
batch_size = 1
epoches = 4
lr = 0.001
class_num = 3

In [ ]:
# dataset 
train_dataset = TensorDataset(x_train, y_train)
valid_dataset = TensorDataset(x_valid, y_valid)

# dataloader
train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=batch_size, drop_last=True)
valid_dataloader = DataLoader(valid_dataset, shuffle=True, batch_size=batch_size, drop_last=False)

# Model


In [ ]:
# pretrained resnet18
class resnet(nn.Module):
    def __init__(self, is_freezed=False):
        super(resnet, self).__init__()
        self.model = models.resnet18(weights='IMAGENET1K_V1')
        if is_freezed:
            for child in self.model.children():
                child.requires_grad = False
        self.model.fc = nn.Sequential(
            nn.Linear(self.model.fc.in_features, 256),
#             nn.BatchNorm1d(256),
            nn.Linear(256, class_num)
        )
    def forward(self, x):
        return self.model(x)

In [ ]:
def make_history():
    history = {}
    history['train_acc'] = []
    history['train_loss'] = []
    history['valid_acc'] = []
    history['valid_loss'] = []
    return history

In [ ]:
# train model
def train_model(model, dataloader, optimizer, loss_fn, history):
    acc = 0
    loss = 0
    model.train()
    for X, y in dataloader:
        X = X.cuda()
        y = y.cuda()
        pred = model(X)
        cost = loss_fn(y, pred)

        optimizer.zero_grad()
        cost.backward()
        optimizer.step()

        acc += torch.sum(torch.argmax(pred, dim=1)==y).item()
        loss += cost.item()
    history['train_acc'].append(acc / len(dataloader.dataset))
    history['train_loss'].append(loss)
    return model, optimizer

def valid_model(model, dataloader, loss_fn, history):
    acc = 0
    loss = 0
    model.eval()
    for X, y in dataloader:
        X = X.cuda()
        y = y.cuda()
        with torch.no_grad():
            pred = model(X)
            cost = loss_fn(y, pred)

        acc += torch.sum(torch.argmax(pred, dim=1)==y).item()
        loss += cost.item()
    history['valid_acc'].append(acc / len(dataloader.dataset))
    history['valid_loss'].append(loss)

In [ ]:
def loss_mse(y, pred):
    y_onehot = nn.functional.one_hot(y, num_classes=class_num)
    loss = torch.mean((pred - y_onehot)**2)
    return loss

In [ ]:
history = make_history()

In [ ]:
from torch.optim import Adam

model = resnet(is_freezed=True).cuda()
optimizer = Adam(params=model.parameters(), lr=lr)
for epoch in tqdm(range(epoches)):
    model, optimizer = train_model(model, train_dataloader, optimizer, loss_mse, history)
    with torch.no_grad():
        valid_model(model, valid_dataloader, loss_mse, history)


In [ ]:
plt.plot(history['train_acc'])
plt.plot(history['valid_acc'])
plt.title('accuracy')
plt.show()

In [ ]:
plt.plot(history['train_loss'])
plt.plot(history['valid_loss'])
plt.title('loss')
plt.show()

In [ ]:
def test_model(model, dataloader):
    y_test_pred = []
    model.eval()
    for X in dataloader:
        X = X[0].cuda()
        pred = model(X)
        
        y_test_pred.append(torch.argmax(pred, dim=1).item())
    return y_test_pred

In [ ]:
test_dataset = TensorDataset(x_valid)
with torch.no_grad():
    test_dataloader = DataLoader(test_dataset, shuffle=False, batch_size=1, drop_last=False)

In [ ]:
y_test_pred = test_model(model, test_dataloader)

In [ ]:
y_test_pred

In [ ]:
torch.save(model.state_dict(), 'model_3_b1.pth')

In [ ]:
device = torch.device('cpu')
model_65 = resnet(is_freezed=True)
model_65.load_state_dict(torch.load('/kaggle/working/model_3_b1.pth', map_location=device))

In [ ]:
seed = 3
k = np.random.randint(len(x_valid))
print(k)
pred = model_65(x_valid[k].reshape(-1, 3, 48, 48))
print(torch.argmax(pred, dim=1))
print(y_valid[k])